In [15]:
import os
import cv2
import json
import random
from pathlib import Path
from sklearn.model_selection import train_test_split

def extract_frames_and_labels(video_path, traco_path, output_img_dir, output_lbl_dir, bbox_size=40):
    cap = cv2.VideoCapture(video_path)
    name = Path(video_path).stem
    frame_map = {}

    with open(traco_path, 'r') as f:
        data = json.load(f)
        for item in data['rois']:
            z = item['z']
            x, y = item['pos']
            frame_map.setdefault(z, []).append((x, y))  # 支持多个目标

    frame_idx = 0
    success, frame = cap.read()
    saved_frames = []

    while success:
        h, w = frame.shape[:2]
        frame_name = f"{name}_{frame_idx:04d}"
        img_path = output_img_dir / f"{frame_name}.jpg"
        label_path = output_lbl_dir / f"{frame_name}.txt"

        cv2.imwrite(str(img_path), frame)
        saved_frames.append(frame_name)

        with open(label_path, 'w') as f:
            if frame_idx in frame_map:
                for cx, cy in frame_map[frame_idx]:
                    bbox_w = bbox_h = bbox_size
                    x_center = cx / w
                    y_center = cy / h
                    w_norm = bbox_w / w
                    h_norm = bbox_h / h
                    f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        success, frame = cap.read()
        frame_idx += 1

    cap.release()
    return saved_frames


def split_and_copy(dataset_root, saved_frames, train_ratio=0.8):
    img_dir = dataset_root / "images"
    lbl_dir = dataset_root / "labels"

    train_imgs = dataset_root / "images" / "train"
    val_imgs   = dataset_root / "images" / "val"
    train_lbls = dataset_root / "labels" / "train"
    val_lbls   = dataset_root / "labels" / "val"

    for d in [train_imgs, val_imgs, train_lbls, val_lbls]:
        d.mkdir(parents=True, exist_ok=True)

    train_list, val_list = train_test_split(saved_frames, train_size=train_ratio, random_state=42)

    for fname in train_list:
        os.rename(img_dir / f"{fname}.jpg", train_imgs / f"{fname}.jpg")
        os.rename(lbl_dir / f"{fname}.txt", train_lbls / f"{fname}.txt")

    for fname in val_list:
        os.rename(img_dir / f"{fname}.jpg", val_imgs / f"{fname}.jpg")
        os.rename(lbl_dir / f"{fname}.txt", val_lbls / f"{fname}.txt")

    return train_list, val_list

def generate_data_yaml(dataset_root, yaml_path="data.yaml"):
    content = f"""train: {dataset_root}/images/train
val: {dataset_root}/images/val

nc: 1
names: ["hexbug"]
"""
    with open(yaml_path, 'w') as f:
        f.write(content)
    print(f"✅ 生成 data.yaml 到 {yaml_path}")

def build_dataset(videos_dir, traco_dir, dataset_root="hexbug_dataset0", bbox_size=40):
    dataset_root = Path(dataset_root)
    img_dir = dataset_root / "images"  # 临时保存所有帧
    lbl_dir = dataset_root / "labels"
    img_dir.mkdir(parents=True, exist_ok=True)
    lbl_dir.mkdir(parents=True, exist_ok=True)

    all_frames = []

    for fname in sorted(os.listdir(videos_dir)):
        if fname.endswith(".mp4"):
            stem = Path(fname).stem
            video_path = os.path.join(videos_dir, fname)
            # 支持 .traco, .json, .traco.sec 三种后缀
            traco_path = None
            for ext in [".traco", ".json", ".traco.sec"]:
                candidate = os.path.join(traco_dir, stem + ext)
                if os.path.exists(candidate):
                   traco_path = candidate
                   break
            if traco_path is None:
               print(f"⚠️ 缺失标注文件: {stem}.traco/.json/.traco.sec")
               continue
 


            if not os.path.exists(traco_path):
                print(f"⚠️ 缺失 traco: {traco_path}")

                continue
            frames = extract_frames_and_labels(video_path, traco_path, img_dir, lbl_dir, bbox_size)
            all_frames.extend(frames)

    train_list, val_list = split_and_copy(dataset_root, all_frames)
    generate_data_yaml(dataset_root)
    print(f"✅ 数据集制作完成，训练样本: {len(train_list)}，验证样本: {len(val_list)}")

# === 调用 ===
build_dataset(
    videos_dir="training/training-Copy1",    # 视频目录
    traco_dir="training/training-Copy1",     # .traco文件目录
    dataset_root="hexbug_dataset0",  # 输出根目录


    bbox_size=40                # 可调大小
)


✅ 生成 data.yaml 到 data.yaml
✅ 数据集制作完成，训练样本: 7345，验证样本: 1837


In [12]:
import os

# 1. 设置目标目录（可修改为你自己的路径）
target_dir = r"training/training-Copy1"  # Windows 示例路径

# 2. 遍历该目录下的所有文件
for filename in os.listdir(target_dir):
    if "..traco.sec" in filename:
        old_path = os.path.join(target_dir, filename)
        new_filename = filename.replace("..traco.sec", ".traco.sec")
        new_path = os.path.join(target_dir, new_filename)
        
        # 执行重命名
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} ➜ {new_filename}")


Renamed: training070..traco.sec ➜ training070.traco.sec
Renamed: training081..traco.sec ➜ training081.traco.sec
Renamed: training072..traco.sec ➜ training072.traco.sec
Renamed: training050..traco.sec ➜ training050.traco.sec
Renamed: training059..traco.sec ➜ training059.traco.sec
Renamed: training041..traco.sec ➜ training041.traco.sec
Renamed: training074..traco.sec ➜ training074.traco.sec
Renamed: training092..traco.sec ➜ training092.traco.sec
Renamed: training040..traco.sec ➜ training040.traco.sec
Renamed: training063..traco.sec ➜ training063.traco.sec
Renamed: training060..traco.sec ➜ training060.traco.sec
Renamed: training075..traco.sec ➜ training075.traco.sec
Renamed: training064..traco.sec ➜ training064.traco.sec
Renamed: training07..traco.sec ➜ training07.traco.sec
Renamed: training096..traco.sec ➜ training096.traco.sec
Renamed: training099..traco.sec ➜ training099.traco.sec
Renamed: training094..traco.sec ➜ training094.traco.sec
Renamed: training054..traco.sec ➜ training054.trac

In [ ]:
from ultralytics import YOLO

# 加载 YOLOv8n 模型（nano 版本，速度快，适合小目标）
model = YOLO("yolov11n.pt")

# 开始训练
model.train(
    data="/home/hpc/tovl/tovl108v/hexbug_dataset0/data.yaml",
    epochs=150,
    imgsz=768,
    batch=16,
    name="hexbug_yolov11n",
    lr0=0.001,
    patience=30,
    augment=True
)

In [ ]:
yolo task=detect mode=val model=runs/detect/hexbug_yolov8n5/weights/best.pt data=your_dataset.yaml

In [3]:
from ultralytics import YOLO

# ✅ 加载你要训练的 YOLOv8n 模型（yolov11n.pt 应是你自定义的权重文件名）
model = YOLO("runs/detect/hexbug_yolo11n3/weights/last.pt")

#恢复断点83/150
model.train(resume=True, device=0)


# ✅ 开始训练
#model.train(  
#    data="/home/hpc/tovl/tovl108v/hexbug_dataset0/data.yaml",  # 数据集路径
#    epochs=150,                         # 训练轮次


#    imgsz=768,                          # 输入图像尺寸（建议 >=640，768适合小目标）
#    batch=16,                           # 批次大小（可根据显存微调）
#    name="hexbug_yolo11n",             # 输出运行名称（用于保存路径）
#    lr0=0.001,                          # 初始学习率
#    patience=30,                        # 早停策略
#    augment=True,                       # 开启数据增强（默认 True）
#    optimizer='AdamW',                  # 可选优化器：AdamW 比 SGD 更稳定
#    cos_lr=True                         # 使用余弦退火学习率调度（训练更平滑）
#)


Ultralytics 8.3.161 🚀 Python-3.12.10 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/hpc/tovl/tovl108v/hexbug_dataset0/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/hexbug_yolo11n3/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=hexbug_yolo11n3, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, ove

Traceback (most recent call last):
  File "/apps/jupyterhub/jh5.3.0-py3.12/lib/python3.12/multiprocessing/queues.py", line 259, in _feed
    reader_close()
  File "/apps/jupyterhub/jh5.3.0-py3.12/lib/python3.12/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/apps/jupyterhub/jh5.3.0-py3.12/lib/python3.12/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /home/hpc/tovl/tovl108v/hexbug_dataset0/labels/train.cache... 7861 images, 0 backgrounds, 2 corrupt: 100%|██████████| 7861/7861 [00:00<?, ?it/s]

train: /home/hpc/tovl/tovl108v/hexbug_dataset0/images/train/training016_0079.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000127]
train: /home/hpc/tovl/tovl108v/hexbug_dataset0/images/train/training061_0056.jpg: ignoring corrupt image/label: negative label values [-3.e-05]


...

                   all       2347       4389      0.951      0.942      0.962       0.58



66 epochs completed in 3.272 hours.
Optimizer stripped from runs/detect/hexbug_yolo11n3/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/hexbug_yolo11n3/weights/best.pt, 5.5MB

Validating runs/detect/hexbug_yolo11n3/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.12.10 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 74/74 [00:30<00:00,  2.42it/s]


                   all       2347       4389      0.921      0.922      0.959      0.577
              hexbug_0       2347       4389      0.921      0.922      0.959      0.577
Speed: 0.2ms preprocess, 9.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/hexbug_yolo11n3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x728dd63a72c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
from ultralytics import YOLO

# 加载之前训练好的模型（用 best.pt 效果更好）
model = YOLO("runs/detect/hexbug_yolo11n3/weights/best.pt")

# 重新训练（不是 resume，而是重新定义训练设置）
model.train(
    data="/home/hpc/tovl/tovl108v/hexbug_dataset0/data.yaml",
    epochs=100,            # 再训练 100 轮（你可以选 50、100、150 都可以）
    imgsz=960,             # 更高分辨率
    batch=8,               # 视 GPU 显存调整（1080Ti 建议最多 8）
    name="hexbug_yolo11n_finetune_960aug",
    lr0=0.0005,            # 可以适当减小学习率进行微调
    patience=30,
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1
)


Ultralytics 8.3.161 🚀 Python-3.12.10 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/hpc/tovl/tovl108v/hexbug_dataset0/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=runs/detect/hexbug_yolo11n3/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=hexbug_yolo11n_finetune_960aug, nbs=64, nms=False, opset=None, optimize=False, opti

train: Scanning /home/hpc/tovl/tovl108v/hexbug_dataset0/labels/train.cache... 7861 images, 0 backgrounds, 2 corrupt: 100%|██████████| 7861/7861 [00:00<?, ?it/s]

train: /home/hpc/tovl/tovl108v/hexbug_dataset0/images/train/training016_0079.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000127]
train: /home/hpc/tovl/tovl108v/hexbug_dataset0/images/train/training061_0056.jpg: ignoring corrupt image/label: negative label values [-3.e-05]



/home/hpc/tovl/tovl108v/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 9.7±3.2 MB/s, size: 313.9 KB)


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     49/100      4.13G      1.379     0.8846     0.9491          8        960: 100%|██████████| 983/983 [03:55<00:00,  4.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 147/147 [00:22<00:00,  6.51it/s]

                   all       2347       4389      0.956      0.931      0.963      0.575



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     50/100      4.13G      1.364     0.8715     0.9383         25        960:   1%|▏         | 13/983 [00:03<03:50,  4.22it/s]

In [1]:
from ultralytics import YOLO

# 加载断点的模型路径（使用上一次的训练目录）
model = YOLO("runs/detect/hexbug_yolo11n_finetune_960aug/weights/last.pt")

# 继续训练
model.train(resume=True)

Ultralytics 8.3.161 🚀 Python-3.12.10 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/hpc/tovl/tovl108v/hexbug_dataset0/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=runs/detect/hexbug_yolo11n_finetune_960aug/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=hexbug_yolo11n_finetune_960aug, nbs=64, nms=False, opset=None, optim

train: Scanning /home/hpc/tovl/tovl108v/hexbug_dataset0/labels/train.cache... 7861 images, 0 backgrounds, 2 corrupt: 100%|██████████| 7861/7861 [00:00<?, ?it/s]

train: /home/hpc/tovl/tovl108v/hexbug_dataset0/images/train/training016_0079.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000127]
train: /home/hpc/tovl/tovl108v/hexbug_dataset0/images/train/training061_0056.jpg: ignoring corrupt image/label: negative label values [-3.e-05]



/home/hpc/tovl/tovl108v/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 14.9±10.6 MB/s, size: 313.9 KB)


...

                   all       2347       4389      0.962      0.953      0.975      0.607



51 epochs completed in 3.718 hours.
Optimizer stripped from runs/detect/hexbug_yolo11n_finetune_960aug/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/hexbug_yolo11n_finetune_960aug/weights/best.pt, 5.5MB

Validating runs/detect/hexbug_yolo11n_finetune_960aug/weights/best.pt...
Ultralytics 8.3.161 🚀 Python-3.12.10 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11165MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 147/147 [00:42<00:00,  3.42it/s]


                   all       2347       4389       0.93      0.937      0.969      0.599
              hexbug_0       2347       4389       0.93      0.937      0.969      0.599
Speed: 0.3ms preprocess, 14.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/hexbug_yolo11n_finetune_960aug


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d79481b8f50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 